# Get Time Series Data From Timestamps

In [2]:
import boto3
import json
import datetime
import pandas as pd
from io import BytesIO
import importlib

IMU_BUCKET = 'matt3r-imu-us-west-2'
s3_client = boto3.client('s3')

In [11]:
# find timestamps to use as a test set
events = []
start_time = datetime.datetime(2023, 7, 25, 17, 0, 0, tzinfo=datetime.timezone.utc).timestamp()
for i in range(10):
    events.append((start_time + i*120, start_time + i*120 + 5))

1690304400.0

In [13]:
# convert timestamp to date and hour
file_names = set()
for event in events:
    start_timestamp = event[0]
    dt_object = datetime.datetime.fromtimestamp(start_timestamp)
    file_names.add(dt_object.strftime("%Y-%m-%d_%H"))

file_names

{'2023-07-25_10'}